In [ ]:
#pip install scikit-learn
#pip install lightgbm

In [1]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report

In [2]:
data = pd.read_csv("/Users/farhat/Documents/Project/ProcessedData/fullData.csv")
data.head(10)

,Patient_Id,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,...,sig_30,sig_31,sig_32,sig_33,sig_34,sig_35,sig_36,sig_37,sig_38,SepsisLabel
0,p016161,74.0,100.0,37.11,NaN,101.0,NaN,14.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,p016161,72.0,98.0,37.11,NaN,81.0,NaN,14.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,p016161,75.0,100.0,37.11,NaN,83.0,NaN,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,p016161,63.0,100.0,37.11,NaN,64.0,NaN,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,p016161,61.0,100.0,37.17,NaN,66.0,NaN,14.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,p016161,68.0,100.0,37.17,NaN,63.0,NaN,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,p016161,64.0,100.0,37.17,NaN,94.0,NaN,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,p016161,66.0,100.0,37.17,NaN,83.0,NaN,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8,p019252,80.0,100.0,34.45,109.5,89.0,72.0,12.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,p019252,80.0,100.0,35.70,90.0,68.0,54.0,10.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [19]:
X = data.iloc[:,:-1]
y = data.iloc[:,[0,-1]]
X.head()

,Patient_Id,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,...,sig_29,sig_30,sig_31,sig_32,sig_33,sig_34,sig_35,sig_36,sig_37,sig_38
0,p016161,74.0,100.0,37.11,NaN,101.0,NaN,14.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p016161,72.0,98.0,37.11,NaN,81.0,NaN,14.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,p016161,75.0,100.0,37.11,NaN,83.0,NaN,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,p016161,63.0,100.0,37.11,NaN,64.0,NaN,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,p016161,61.0,100.0,37.17,NaN,66.0,NaN,14.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Assuming you have X (features) and y (target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)

(1264344, 83)
(316086, 83)
(1264344, 2)
(316086, 2)


In [24]:
print(y_train.sort_index())

        Patient_Id  SepsisLabel
0          p016161            1
1          p016161            1
3          p016161            1
4          p016161            1
5          p016161            1
...            ...          ...
1580423    p011617            0
1580424    p011617            0
1580425    p011617            0
1580426    p011617            0
1580428    p011617            0

[1264344 rows x 2 columns]


In [11]:
# LightGBM Dataset
train_data = lgb.Dataset(X_train, label=y_train)

# Parameters
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbose': -1
}

# Train the model
model = lgb.train(params, train_data, num_boost_round=50)

In [12]:
y_pred = model.predict(X_test)
y_pred = (y_pred >= 0.5).astype(int)

y_pred

array([0, 0, 0, ..., 0, 0, 0], shape=(316086,))

In [13]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9807773833703486
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    309265
           1       0.79      0.15      0.25      6821

    accuracy                           0.98    316086
   macro avg       0.89      0.57      0.62    316086
weighted avg       0.98      0.98      0.97    316086



In [14]:
from sklearn.metrics import confusion_matrix

In [15]:
confusion_matrix(y_test,y_pred=y_pred)

array([[308996,    269],
       [  5807,   1014]])

In [25]:
np.unique(y_test, return_counts=True)

(array([0, 1]), array([154566,   3477]))

In [21]:
print (y_test)
print (y_pred)

70024     0
181934    0
724581    0
199837    0
581915    0
         ..
665469    0
452072    0
493677    0
536585    0
405136    0
Name: SepsisLabel, Length: 158043, dtype: int64
[0 0 0 ... 0 0 0]


In [22]:
pd.DataFrame({
    'Sepsis Label' : y_test,
    'Sepsis Predicted':y_pred
})

,Sepsis Label,Sepsis Predicted
70024,0,0
181934,0,0
724581,0,0
199837,0,0
581915,0,0
...,...,...
665469,0,0
452072,0,0
493677,0,0
536585,0,0


In [ ]:
dtrain = lgb.Dataset(X_train, label=y_train)

params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbose': -1
}

# 5-fold CV
cv_results = lgb.cv(params, dtrain, num_boost_round=100, nfold=5, stratified=True, shuffle=True, seed=42)

# View results
print("Best log loss:", cv_results.keys())


Best log loss: dict_keys(['valid binary_logloss-mean', 'valid binary_logloss-stdv'])
